# Backbone of the app

## 0. Load the tiff images (to grayscale?)
   **IN** tiff file name (tiff file?)
   
   *path --> im_stack --> im list*
   
   **OUT** list of images

In [8]:
import tifffile
import numpy as np
import cv2
import matplotlib.pyplot as plt

def load_tiff_images(tiff_path):
    image_stack = tifffile.imread(tiff_path)
    return list(image_stack)

## 1. preprocess the image depending on- blur kernel (?), dilate (?) threshold value
   **IN** opencv image, threshold value, dilation value...
   
   *im -->  blur(?) --> dilate(?) --> threshold*
   
   **OUT** binary image, with dilated areas

In [193]:
def blur_im(img, kernel_size=3):
    blur = cv2.GaussianBlur(img, (kernel_size, kernel_size),0)
    return blur

def dilate_image(img, kernel_size=3):
    kernel = np.ones((kernel_size,kernel_size))
    kernel[0,0], kernel[0, kernel_size-1], kernel[kernel_size-1, 0],kernel[kernel_size-1, kernel_size-1] = 0,0,0,0 
    kernel = kernel.astype(np.uint8)
    dilated = cv2.dilate(img, kernel, iterations=1)
    return dilated

def threshold_image(img, threshold_value):
    ret, thresh = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)
    return thresh

def preprocess_image(img, threshold_value, kernel_size=3 ):
    #todo: genrelize to N functions in order, multiple kernels
    return threshold_image(dilate_image(blur_im(img, kernel_size), kernel_size), threshold_value)

## 2. Find contours and store & present each sepparatley (on the main image?)

In [192]:
def get_contours(img, min_area=0):
    img = img.astype(np.uint8)
    contours, heir = cv2.findContours(image=img, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
    contours = [con for con in contours if cv2.contourArea(con)> min_area]
    return contours

def draw_contours(img, contours, thick=3):
#     todo: more space to play with the drawing
    im_copy = img.copy()
    cv2.drawContours(image=im_copy, contours=contours, contourIdx=-1, color=(255,0,0), thickness=7)
    cv2.imwrite("p1.png", im_copy)
    return im_copy

def get_contour_patch(img, contour):
    x, y, w, h  = cv2.boundingRect(contour)
    return img[ y:y+h, x:x+w] 


## 3. scan other channels in each contour for contours!

# TODO
* think of an abstraction variable for 'smaller?' 'less fine?'
* let the user define which channel to start with
* ability to get rid from a contour
* understand tiff file